In [1]:
import os, pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder

Calculating the total amount of available images

In [2]:
nb_batches = 5
nb_images = 0
for i in range(nb_batches):
    with open("pickles/labels"+str(i)+".pickle", "rb") as file:
        labels = pickle.load(file)
        nb_images += len(labels)
print(nb_images, "images will be used to train the network")

9805 images will be used to train the network


### Neural network part

In [3]:
#Clearing past graphs
tf.reset_default_graph()   

learning_rate = 0.02

#placeholders definition
input_data = tf.placeholder(tf.float32, (None, 128, 128, 3))
input_labels = tf.placeholder(tf.int32, (None, 28))
is_training = tf.placeholder(tf.bool)

#layers definition
conv1 = tf.layers.conv2d(input_data, 16, (5, 5), (1, 1), activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1, (2, 2), (2, 2))
add = tf.layers.

conv2 = tf.layers.conv2d(pool1, 16, (5, 5), (1, 1), activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, (2, 2), (2, 2))

conv3 = tf.layers.conv2d(pool2, 64, (3, 3), (1, 1), activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(conv3, (2, 2), (2, 2))

conv4 = tf.layers.conv2d(pool3, 64, (3, 3), (1, 1), activation=tf.nn.relu)
pool4 = tf.layers.max_pooling2d(conv4, (2, 2), (2, 2))

flattened = tf.contrib.layers.flatten(pool4)

# define fully-connected (or dense) layers
dense1 = tf.layers.dense(flattened, 1000)
drop1 = tf.layers.dropout(dense1, rate=0.5, training=is_training)

dense2 = tf.layers.dense(drop1, 300)
drop2 = tf.layers.dropout(dense2, rate=0.5, training=is_training)

dense3 = tf.layers.dense(drop2, 100)
drop3 = tf.layers.dropout(dense3, rate=0.5, training=is_training)

logits = tf.layers.dense(drop3, 28)

# define loss and training operation
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=input_labels, logits=logits))
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# calcualte average accuracy over a batch of images
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(input_labels, 1)), tf.float32))
good_predictions = tf.cast(tf.reduce_sum(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(input_labels, 1)), tf.float32)), tf.float32)

In [4]:
#defining parameters of training run
epoch = 40
sub_batch_size = 100
nb_steps = nb_images * epoch // sub_batch_size
log_frequency = 50
batch_range = [i for i in range(nb_batches)]
batch_indices = []
for i in range(epoch):
    batch_indices += batch_range
    
print(nb_steps, "steps will be performed.")
    
with tf.Session() as sess:
    #Initialization
    sess.run(tf.global_variables_initializer())
    batch_run = 0
    
    for step_index in range(nb_steps):
        #On first step
        if step_index == 0:
            with open("pickles/images0.pickle", "rb") as file:
                images = pickle.load(file)
            with open("pickles/labels0.pickle", "rb") as file:
                labels = pickle.load(file)
            sub_batch_index = 0
            batch_size = len(labels)
                
        #If we are crossing images sets
        elif (sub_batch_index + 1)* sub_batch_size > batch_size:
            before_crossing_images = images[-(batch_size - (sub_batch_index-1)*sub_batch_size):]
            before_crossing_labels = labels[-(batch_size - (sub_batch_index-1)*sub_batch_size):]
            with open("pickles/images"+str(batch_indices[batch_run])+".pickle", "rb") as file:
                images = np.concatenate((before_crossing_images, pickle.load(file)))
            with open("pickles/labels"+str(batch_indices[batch_run])+".pickle", "rb") as file:
                labels = np.concatenate((before_crossing_labels, pickle.load(file)))
            sub_batch_index = 0
            batch_size = len(labels)
            batch_run += 1
            
        indices = [i for i in range(sub_batch_size*sub_batch_index, sub_batch_size*(sub_batch_index+1))]
        
        #performing training
        batch_loss, batch_accuracy, _ = sess.run([loss, accuracy, train_op], feed_dict={
            input_data: images.take(indices, axis=0),
            input_labels: labels.take(indices, axis=0),
            is_training: True
            })
            
        #logging
        if step_index % log_frequency == 0:
            print('step:', step_index, ', loss:', batch_loss, ', training accuracy:', batch_accuracy)
        
        sub_batch_index += 1
        
    ### END OF TRAINING
    
    print("Training finished.")
    
    with open("pickles/images6.pickle", "rb") as file:
        images = pickle.load(file)
    with open("pickles/labels6.pickle", "rb") as file:
        labels = pickle.load(file)
        
    test_size = len(labels)
    sub_batch_size = 20
    total_correct = 0
    
    for step in range(test_size // sub_batch_size):
        offset = step * sub_batch_size
        batch_data = images[offset:(offset + sub_batch_size)]
        batch_labels = labels[offset:(offset + sub_batch_size)]
        
        corrects = sess.run(good_predictions,feed_dict={
            input_data: batch_data,
            input_labels: batch_labels,
            is_training: False
        })
        total_correct += corrects
        
    test_accuracy = 100.0 * (total_correct / test_size)

    print("Validation accuracy: {0:.2f}%.".format(test_accuracy))

9805 steps will be performed.
step: 0 , loss: 4.0000954 , training accuracy: 0.04
step: 50 , loss: 2.9077897 , training accuracy: 0.15
step: 100 , loss: 2.8013184 , training accuracy: 0.14
step: 150 , loss: 2.8902655 , training accuracy: 0.12
step: 200 , loss: 2.7035992 , training accuracy: 0.2
step: 250 , loss: 2.383393 , training accuracy: 0.29
step: 300 , loss: 2.6205566 , training accuracy: 0.21
step: 350 , loss: 2.6164308 , training accuracy: 0.16
step: 400 , loss: 2.5116537 , training accuracy: 0.27
step: 450 , loss: 2.4691286 , training accuracy: 0.21
step: 500 , loss: 2.4649503 , training accuracy: 0.26
step: 550 , loss: 2.4429471 , training accuracy: 0.27
step: 600 , loss: 2.1128292 , training accuracy: 0.32
step: 650 , loss: 2.3187685 , training accuracy: 0.28
step: 700 , loss: 2.2526731 , training accuracy: 0.28
step: 750 , loss: 2.2106674 , training accuracy: 0.32
step: 800 , loss: 2.4697838 , training accuracy: 0.22
step: 850 , loss: 2.4262679 , training accuracy: 0.23
ste

step: 7500 , loss: 0.3806504 , training accuracy: 0.89
step: 7550 , loss: 0.72362113 , training accuracy: 0.76
step: 7600 , loss: 0.6316234 , training accuracy: 0.83
step: 7650 , loss: 0.5271745 , training accuracy: 0.8
step: 7700 , loss: 0.5670472 , training accuracy: 0.84
step: 7750 , loss: 0.6252847 , training accuracy: 0.81
step: 7800 , loss: 0.5468813 , training accuracy: 0.82
step: 7850 , loss: 0.47613975 , training accuracy: 0.84
step: 7900 , loss: 1.8097408 , training accuracy: 0.5
step: 7950 , loss: 0.6186912 , training accuracy: 0.74
step: 8000 , loss: 0.6189581 , training accuracy: 0.82
step: 8050 , loss: 0.79415923 , training accuracy: 0.76
step: 8100 , loss: 0.4114903 , training accuracy: 0.83
step: 8150 , loss: 0.46230522 , training accuracy: 0.81
step: 8200 , loss: 0.37399998 , training accuracy: 0.9
step: 8250 , loss: 0.47374997 , training accuracy: 0.81
step: 8300 , loss: 0.6144824 , training accuracy: 0.77
step: 8350 , loss: 0.8974759 , training accuracy: 0.68
step: 8